# Redes Neuronales y como grokearlas.

Si no estas aun familiarizado con el funcionamiento de las redes neurales puedes que te cause timidez la matematica y notacion aparentemente complicada ademas del nombre de ciencia ficción. Pero no temas, aqui va una introducción intuitiva (y baja en las matematicas) sobre el modelo de redes neuronales. Empezemos con algo pequeño: 

## Circuito de operaciones: 

Empeze a entender a las redes neuronales cuando me las presentaron como un circuito de operaciones que regresan valores **reales** en lugar de valores booleanos. Con esta analogia, los valores generados por nuestras compuertas fluyen hacia adelante en nustro circuito con compuertas binarias y unarias representados por operadores como: **-**, **+**, **exp**, etc...

Exploremos el siguiente ejemplo: 

<img src="files/images/multGate.png" style="width:320px;heigth:320px;">

Este circuito recibe dos valores de entrada (**x**, **y**) y evalua la operacion **x * y **. Este es el comportamiento de la compuerta ** * **. Todas las compuertas que podemos incluir en nuestro circuito se comportaran de manera analoga. Toman 1 o 2 valores de entrada y devuelven un unico valor de salida. 

La implementación de la compuerta de arriba es la siguiente: 

In [1]:
def compuertaMultiplicacion(x, y):
    return x * y;

print compuertaMultiplicacion(3,-4)

-12


Ahora hay que definir el problema que queremos atacar:

1. Le proporcionamos al circuito especificos valores de entrada (ejemplo: ** x = 3 **, ** y = -4 **)
2. El circuito nos devuelve un valor de salida.
3. Como modificamos lijeramente nuestros valores de entrada para aumentar nuestro valor de salida?

Para este circuito la respuesta es aparente, aumenta **x** y **y** hasta el infinito y recibe outputs de tamaño infinito, pero trabajemos de buena fe y asumamos que luego tendremos que lidiar con un circuito compuesta de miles de compuertas realizando operaciones mas complejas con miles de outputs distintos. En ese caso necesitaremos de un algoritmo formal para maximizar nuestro valor de salida.

### Gradiente Numerica.

Tomando como valores de entrada **x=3** y **y=-4** tenemos un valor de salida igual a **-12**. Si quisieramas aumentar este valor una interesante manera de verlo es imaginar que le damos un 'jalon' al valor de salida en la direccion positiva y como este punto de salida esta atado de cierta manera a los valores de entrada veriamos reflejado los efectos de este 'jalon' en ellos.

Esta fuerza que describo resulta ser la **derivada** del valor de salida en respecto a los valores de entrada. En otras palabras podemos interpretas a la derivada respecto a **x** como la fuerza ejercida sobre **x** cuando le damos un jalon a nuestro valor de salida con la meta de aumentarlo.

Podemos escribir la derivada de nuestra función (o circuito) respecto a x de la siguiente forma: 

$$
\frac{\partial f(x,y)}{\partial x} = \frac{f(x+h,y) - f(x,y)}{h}
$$

De el lado izquierdo tenemos la expresion que representa la derivada de nuestra función respecto a **x** y la calculamos tomando la diferencia de output de nuestro circuito con los valores de entrada actuales y nuestro circuito con un lijero ajuste (_h_) en la variable x. Esa diferencia la divideremos sobre el valor de ajuste _h_.

La implementación en codigo es de la sigiuente manera:


In [2]:
# Valores de entrada
x = 3 
y = -4 

# f(x,y), resultado de nuestro circuito
salida = compuertaMultiplicacion(x, y) 

# Valor de ajuste
h = 0.0001 

### Derivada respecto a x
x_step = x + h # 3.0001
salida2 = compuertaMultiplicacion(x_step, y) #-12.004
derivada_x = (salida2 - salida) / h #-4.0

### Derivada respecto a y
y_step = y + h # -3.999
salida3 = compuertaMultiplicacion(x, y_step) #-11.9997
derivada_y = (salida3 - salida) / h # 3.0


Por los resultados de arriba lo que observamos es que aumentar **x** por **h**, disminuyo nuestro valor de salida (_-12.004_). Por lo cual tiene sentido que nuestra derivada en relacion x sea negativa y de magnitud **_-4.0_**. Direccion y magnitud es lo queremos mantener en mente aqui, la derviada en este caso nos dice que para aumentar el resultado de nuestro circuito debemos de ajustar la variable **x** con magnitud **4.0** y direccion **-1**. Por el otro lado la derivada respecto a **y** nos presenta un ajuste con magnitud **3.0** y direccion **+1**.

tl;dr El circuito _quiere_ disminuir **x** con intesidad 4 y quiere aumentar **y** con intensidad 3.

Se les llama derivadas a la funcion derivada respecto a 1 input. Si tomo todas estas derivadas y las meto en un vector (o lista w/e) a ese se le conoce como **Gradiente**.

Ajustemos nuestras variables de entrada segun los resultados de el gradiente: 


In [3]:
step_size = 0.01
salida = compuertaMultiplicacion(x, y)
x = x + step_size * derivada_x
y = y + step_size * derivada_y
salida_nueva = compuertaMultiplicacion(x, y)

print salida, salida_nueva

-12 -11.7512


Sobre **"step_size"**. Es el tamaño de paso que vamos a dar para ajustar nuestra funcion, no siempre es bueno hacerlo lo mas grande posible por que existe el peligro de caer en ciclos y minimos locales.

Una analogía para describir el proceso es la de estar subiendo un cerro con los ojos vendados. Podemos percibir la inclinacion del cerro con nuestros pies y dar un paso pequeño hacia adelante, si tomaramos un paso hacia adelante demasiado grande correomos el riesgo de caer en un barranco.

Algo interesante de notar es que la derivada de **"x"** es igual al valor de entrada **"y"** y vice-versa. Esto no es una coincidencia y es la clave para hacer el calculo de la gradiente una operacion rapidisima aunque tengamos cientos de miles de valores iniciales en nuestro circuito.